In [ ]:
import coi_maps
import fetch
import geopandas as gpd

In [ ]:
# pull csv down from endpoints
ids = "https://o1siz7rw0c.execute-api.us-east-2.amazonaws.com/beta/submissions/districtr-ids/michigan"
csv_url = "https://o1siz7rw0c.execute-api.us-east-2.amazonaws.com/beta/submissions/csv/michigan"
plan = csv_url + "?type=plan&length=10000"
cois = csv_url + "?type=coi&length=10000"
written = csv_url + "?type=written&length=10000"

_, coi_df, _ = fetch.submissions(ids, plan, cois, written)

In [ ]:
cois = coi_maps.assignment_to_shape(coi_df)

In [ ]:
cois.head()

In [ ]:
len(set(cois['id']))

In [ ]:
dissolved = cois.dissolve('id')

In [ ]:
dissolved.boundary.plot()

In [ ]:
dissolved = dissolved.drop('tile_id', axis = 1)
outfile = "../Michigan/mi_cois.shp"
dissolved.to_file(outfile)

In [ ]:
dissolved.head()

In [ ]:
len(dissolved)